In [ ]:
import os
from glob import glob
import pathlib
import shutil

import earthpy as et
import earthpy.earthexplorer as etee
import earthpy.spatial as es
import geopandas as gpd
import geoviews as gv
import holoviews as hv
import hvplot.pandas
import hvplot.xarray
import numpy as np
import pandas as pd
import panel as pn
import rioxarray as rxr
import rioxarray.merge as rxrm

data_dir = os.path.join(et.io.HOME, et.io.DATA_NAME)
chi_dir = os.path.join(data_dir, 'chicago-neighborhoods')
ndvi_dir = os.path.join(data_dir, 'chicago-green-space', 'processed')

for a_dir in [data_dir, chi_dir, ndvi_dir]:
    if not os.path.exists(a_dir):
        os.makedirs(a_dir)

In [ ]:
# Download neighborhoods shapefile and create GDF of Humboldt Park
chi_path = os.path.join(chi_dir, 'chicago-neighborhoods.shp')
if not os.path.exists(chi_path):
    chi_url = ('https://data.cityofchicago.org/api/geospatial/bbvz-uum9?'
               'method=export&format=Shapefile')
    gpd.read_file(chi_url).to_file(chi_path)

chi_gdf = gpd.read_file(chi_path).set_index('pri_neigh')
neigh_gdf = (
    chi_gdf.
    loc[['Humboldt Park', 'Lincoln Park']]
)
neigh_gdf

In [ ]:
def download_neighborhood_data(name, geometry, start, end):
    """
    Download NAIP raster for a given geometry, start date, and end date

    Parameters
    ==========
    name : str
      The name used to label the download
    geometry : shapely.POLYGON
      The geometry to derive the download extent from. 
      Must have a `.bounds` attribute.
    start : str
      The start date as 'YYYY-MM-DD'
    end : strL
      The end date as 'YYYY-MM-DD'

    Returns
    =======
    downloader : earthpy.earthexplorer.EarthExplorerDownloader
      Object with information about the download, including the data directory.
    """
    
    print(f'Downloading NAIP for {name}')
    bbox = etee.BBox(*geometry.bounds)
    #label = row.pri_neigh.lower().replace(' ', '-')
    naip_downloader = etee.EarthExplorerDownloader(
        dataset="NAIP", 
        label=name.lower().replace(' ', '-'), 
        bbox=bbox,
        start=start, 
        end=end,
        store_credential=True)
    #print(naip_downloader.label)
    #print(naip_downloader.bbox.urx, naip_downloader.bbox.ury)
    naip_downloader.submit_download_request()
    naip_downloader.download(override=False)
    return naip_downloader

In [ ]:
def load_and_merge_arrays(name):
    """
    Load in and merge downloaded arrays
    
    Parameters
    ==========
    name : str
      The name used to label the download

    Returns
    =======
    merge_da : rxr. DataArray
        DataArray with the merged data
    """
    
    print(f'Merging array for {name}')
    data_path = os.path.join(
        et.io.HOME, et.io.DATA_NAME,
        name.lower().replace(' ', '-'))
    tif_paths = glob(os.path.join(data_path, '*.tif'))
    das = [rxr.open_rasterio(tp, masked=True) for tp in tif_paths]
    merged_da = rxrm.merge_arrays(das)
    return merged_da

In [ ]:
def calculate_ndvi_statistics(gdf, da, stats_path, override=False):
    """
    Calculate NDVI, then summarize and save statistics
    
    Uses downloaded National Agricultural Imagery Program (NAIP)
    data.
    
    Parameters
    ==========
    gdf : gpd.GeoDataFrame
      A geodataframe with a single row/neighborhood name/boundary
    da : rxr.DataArray
      Multispectral (NAIP) raster data
    stats_path : pathlike
      Path to the statistics CSV file
    """
    name = str(gdf.index[0])
    print(f'Calculate NDVI and statistics for {name}')
    # Caching for existing data
    file_is_empty = True
    if os.path.exists(stats_path):
        print('Stats file exists.')
        stats_df = pd.read_csv(stats_path)
        file_is_empty = len(stats_df)==0
        print(f'Stats file is empty? {file_is_empty}')
        
        if not file_is_empty:
            if (name in list(stats_df.neighborhood)) and (not override):
                print(f'Stats exist for {name}, skipping')
                return            
            
    reprojected_gdf = gdf.to_crs(da.rio.crs)
    # clip the NAIP tile to the bounds of the neighborhood gdf
    naip_crop_da = da.rio.clip_box(*reprojected_gdf.total_bounds)

    # Clip to extent of neighborhood
    naip_da = naip_crop_da.rio.clip(reprojected_gdf.geometry)

    # Calculate NDVI
    ndvi_da = (
        (naip_da.sel(band=4) - naip_da.sel(band=1))
        / (naip_da.sel(band=4) + naip_da.sel(band=1))
    )

    #Calculate NDVI stats for neighborhood and append to CSV
    print(f'Writing stats for {name}')
    mode = 'w' if file_is_empty else 'a'
    pd.DataFrame(dict(
        neighborhood=[name],
        ndvi_minimum=[float(ndvi_da.min())],
        ndvi_maximum=[float(ndvi_da.max())],
        ndvi_median=[float(ndvi_da.median())],
        ndvi_25pctl=[np.nanpercentile(ndvi_da.values, 25)],
        ndvi_75pctl=[np.nanpercentile(ndvi_da.values, 75)],
        ndvi_mean=[float(ndvi_da.mean())],
        ndvi_stddev=[float(ndvi_da.std())]
        )).to_csv(stats_path, mode='a', header=file_is_empty, index=False)

In [ ]:
ndvi_stats_path = os.path.join(ndvi_dir, 'neighborhood-ndvi-stats.csv')

for neighborhood_name, details in chi_gdf.iterrows():
    if not os.path.exists(ndvi_stats_path):
        print('NDVI stats file does not exist...')
        ndvi_stats_df = pd.DataFrame()
    else:
        ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col="neighborhood")
    if neighborhood_name in ndvi_stats_df.index:
        print(f'Neighborhood stats for {neighborhood_name} have already been calculated. Skipping')
        continue
        
    downloader = download_neighborhood_data(
        neighborhood_name, details.geometry, '2021-01-01', '2021-12-31')
    merged_da = load_and_merge_arrays(neighborhood_name)
    calculate_ndvi_statistics(chi_gdf.loc[[neighborhood_name]], merged_da, ndvi_stats_path)
    del merged_da
    
    # shutil.rmtree(downloader.data_dir)